Convierte lat y lon a UTM

In [1]:
import pandas as pd
import numpy as np

Lectura de datos de movilidad

In [2]:
%%time
data_movilidad = pd.read_parquet('../data_google/data_movilidad.parquet')
#data_movilidad.head()

CPU times: user 43.7 s, sys: 2min 56s, total: 3min 40s
Wall time: 5min


EPSG:4326
WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS

https://gidahatari.com/ih-es/tutorial-para-transformar-sistemas-de-coordenadas-de-puntos-xy-con-python-pandas-y-pyproj

## Proyección de los datos

In [6]:
from pyproj import Transformer
transformer = Transformer.from_crs('epsg:4326','epsg:32612',always_xy=True)

### Convierte lon, lat a UTM

In [7]:
%%time
points = list(zip(data_movilidad.lon, data_movilidad.lat))
coordsUTM = np.array(list(transformer.itransform(points)))

CPU times: user 1min 20s, sys: 17.6 s, total: 1min 38s
Wall time: 2min 2s


Renombra variables

In [8]:
coordsUTM = pd.DataFrame(coordsUTM)
coordsUTM.set_axis(['X', 'Y'], axis='columns', inplace=True)

Agrega a data frame

In [9]:
%%time
data_movilidad.insert(data_movilidad.shape[1], 'X', coordsUTM.X)
data_movilidad.insert(data_movilidad.shape[1], 'Y', coordsUTM.Y)

CPU times: user 137 ms, sys: 301 ms, total: 437 ms
Wall time: 486 ms


In [10]:
#data_movilidad.head()

In [11]:
data_movilidad = data_movilidad.drop(['lat', 'lon'], axis=1)
#data_movilidad.head()

In [12]:
#data_movilidad.to_parquet('../data_google/data_movilidad_UTC_MST.parquet')

### Convierte UTC a MST

In [13]:
from datetime import datetime
from dateutil import tz

from_zone = tz.gettz('UTC')
to_zone = tz.gettz('MST')

In [14]:
def UTC_to_MST(vector, from_zone, to_zone):
    mst = [] #pd.DataFrame
    for utc in vector:
        utc = datetime.strptime(utc, '%Y-%m-%d %H:%M:%S')
        utc = utc.replace(tzinfo=from_zone)
        transf_mst = utc.astimezone(to_zone)
        transf_mst = transf_mst.strftime('%Y-%m-%d %H:%M:%S')
        #transf_mst = '"'+transf_mst+'"'
        mst.append(transf_mst)
    return mst

In [16]:
MST = UTC_to_MST(data_movilidad.timestamp, from_zone, to_zone)

In [17]:
data_movilidad.insert(data_movilidad.shape[1], 'MST', MST)

In [18]:
data_movilidad_transf = data_movilidad[['id_adv', 'X', 'Y', 'MST']]
#data_movilidad_transf.head()

In [19]:
#data_movilidad.head()

In [20]:
#data_movilidad_transf.to_parquet('../data_google/data_movilidad_UTC_MST.parquet')

Crea columna con día y con fecha

In [21]:
#data_movilidad_transf = pd.read_parquet('../data/data_movilidad_UTC_MST.parquet')
#data_movilidad_transf.head()

Prueba para separar fecha de hora

In [23]:
#!pip install regex
import re
B = "2020-09-17 17:09:04"
(dia, hora) = re.split(" ", B)
print(dia)
print(hora)

2020-09-17
17:09:04


In [24]:
dia = []
hora = []

for i in range(0, len(data_movilidad_transf.MST)):
    t = re.split(" ", data_movilidad_transf.MST[i])
    dia.append(t[0])
    hora.append(t[1])

In [25]:
data_movilidad_transf.insert(data_movilidad_transf.shape[1], 'dia', dia)
data_movilidad_transf.insert(data_movilidad_transf.shape[1], 'hora', hora)

In [23]:
#data_movilidad_transf.to_parquet('../data_google/data_movilidad_UTC_MST.parquet', index=False)

## Guardar data frame por mes

In [26]:
%%time
#data_movilidad_transf = pd.read_parquet('../data/data_movilidad_UTC_MST.parquet')
#data_movilidad_transf.head()

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 240 µs


In [9]:
%%time
data_09 = data_movilidad_transf[data_movilidad_transf['dia'].str.contains('2020-09')]
data_09.to_parquet('../data_google/data_movilidad_UTC_MST_09.parquet', index=False)

CPU times: user 17.5 s, sys: 2.26 s, total: 19.8 s
Wall time: 20.9 s


In [4]:
%%time
data_10 = data_movilidad_transf[data_movilidad_transf['dia'].str.contains('2020-10')]
#data_10.to_parquet('../data/data_movilidad_UTC_MST_10.parquet', index=False)

CPU times: user 25.3 s, sys: 4.48 s, total: 29.8 s
Wall time: 32.5 s


In [5]:
%%time
data_11 = data_movilidad_transf[data_movilidad_transf['dia'].str.contains('2020-11')]
#data_11.to_parquet('../data/data_movilidad_UTC_MST_11.parquet', index=False)

CPU times: user 22.6 s, sys: 4.04 s, total: 26.6 s
Wall time: 29.8 s


In [6]:
%%time
data_12 = data_movilidad_transf[data_movilidad_transf['dia'].str.contains('2020-12')]
#data_12.to_parquet('../data/data_movilidad_UTC_MST_12.parquet', index=False)

CPU times: user 17.5 s, sys: 2.36 s, total: 19.9 s
Wall time: 22.2 s


In [11]:
data_09.shape[0] + data_10.shape[0] + data_11.shape[0] + data_12.shape[0]

47592840

In [70]:
#datetime.strptime(data_movilidad_transf.MST[0], '%Y-%m-%d %H:%M:%S')

datetime.datetime(2020, 9, 17, 17, 9, 4)